In [1]:
# import tools

import pandas as pd
import numpy as np
import os
import ast
import json

In [17]:
# raw label load(train)

# raw_normal = pd.read_csv('./data_raw/train_label/sampled_df/sampled_normal_20.csv', encoding='utf-8')
# raw_blight = pd.read_csv('./data_raw/train_label/sampled_df/sampled_blight_20.csv', encoding='utf-8')
# raw_wilt = pd.read_csv('./data_raw/train_label/sampled_df/sampled_wilt_20.csv', encoding='utf-8')
# raw_scorch = pd.read_csv('./data_raw/train_label/sampled_df/sampled_scorch_20.csv', encoding='utf-8')
# raw_chlorosis = pd.read_csv('./data_raw/train_label/sampled_df/sampled_chlorosis_20.csv', encoding='utf-8')

vl_raw_normal = pd.read_csv('./data_raw/test_label/merged_df/vl_raw_en_normal.csv', encoding='utf-8')
vl_raw_blight = pd.read_csv('./data_raw/test_label/merged_df/vl_raw_en_blight.csv', encoding='utf-8')
vl_raw_wilt = pd.read_csv('./data_raw/test_label/merged_df/vl_raw_en_wilt.csv', encoding='utf-8')
vl_raw_scorch = pd.read_csv('./data_raw/test_label/merged_df/vl_raw_en_scorch.csv', encoding='utf-8')
vl_raw_chlorosis = pd.read_csv('./data_raw/test_label/merged_df/vl_raw_en_chlorosis.csv', encoding='utf-8')

tr_normal = pd.read_csv('./data_raw/train_label/merged_df/raw_en_normal.csv', encoding='utf-8')
tr_blight = pd.read_csv('./data_raw/train_label/merged_df/raw_en_blight.csv', encoding='utf-8')
tr_wilt = pd.read_csv('./data_raw/train_label/merged_df/raw_en_wilt.csv', encoding='utf-8')
tr_scorch = pd.read_csv('./data_raw/train_label/merged_df/raw_en_scorch.csv', encoding='utf-8')
tr_chlorosis = pd.read_csv('./data_raw/train_label/merged_df/raw_en_chlorosis.csv', encoding='utf-8')

C:\Users\KM_AI_90\AppData\Local\Temp\ipykernel_1948\442082400.py:15: DtypeWarning: Columns (30,31,32,35,41) have mixed types. Specify dtype option on import or set low_memory=False.
  tr_normal = pd.read_csv('./data_raw/train_label/merged_df/raw_en_normal.csv', encoding='utf-8')


In [18]:
# select columns
col_selection = ['environments', 'annotations', 'categories', 'images.fname', 'images.width',
                 'images.height', 'images.disease_class', 'images.disease_cause_method', 'folder_path']

# normal_data = raw_normal[col_selection].copy()
# blight_data = raw_blight[col_selection].copy()
# wilt_data = raw_wilt[col_selection].copy()
# scorch_data = raw_scorch[col_selection].copy()
# chlorosis_data = raw_chlorosis[col_selection].copy()

vl_normal_data = vl_raw_normal[col_selection].copy()
vl_blight_data = vl_raw_blight[col_selection].copy()
vl_wilt_data = vl_raw_wilt[col_selection].copy()
vl_scorch_data = vl_raw_scorch[col_selection].copy()
vl_chlorosis_data = vl_raw_chlorosis[col_selection].copy()

In [4]:
def columns_selection(df, col_list):
    df_sel = df[col_list].copy()
    return df_sel

In [7]:
# print(blight_data['images.disease_cause_method'].unique())
# print(wilt_data['images.disease_cause_method'].unique())
# print(scorch_data['images.disease_cause_method'].unique())
# print(chlorosis_data['images.disease_cause_method'].unique())

['역병주입']
['수분제한60%']
['EC 3.0' 'EC 0.75' 'pH 5.0' 'pH 7.5' '칼슘부족']
['철결핍' '질소부족']


In [5]:
## label preprocess functions

def json_data_to_pandas(df, target_col, saving_col):
    df[saving_col] = None
    
    for idx, row in df.iterrows():
        json_str = row[target_col]
        str_corrected = json_str.replace("None", "null").replace("'", "\"")
        corrected_data = json.loads(str_corrected)
        df.at[idx, saving_col] = corrected_data

    dict_format = df[saving_col][0]
    extraction_keys = dict_format[0].keys()
    key_list = list(extraction_keys)
    df_pandas_type = df.drop([target_col], axis=1)

    print(type(df[saving_col]))
    return df_pandas_type, key_list

def extraction_env_values(df_pandas_type, key_list, target_col):
    df = df_pandas_type
    df['value_types'] = None
    value_names = []

    for i in range(len(key_list)):
        candidate = key_list[i]
        if candidate.endswith('_value'):
            value_names.append(candidate)

    for idx, row in df.iterrows():
        value_dict = {}
        dict_list = row[target_col]
        for dictionary in dict_list:
            for key, value in dictionary.items():
                if key in value_names:
                    if key not in value_dict:
                        value_dict[key] = [value]
                    else:
                        value_dict[key].append(value)
        df.at[idx, 'value_types'] = value_dict
    
    for idx, row in df.iterrows():
        value_to_split = row['value_types']
        for key, value in value_to_split.items():
            if key not in df.columns:
                df[key] = pd.NA
            df.at[idx, key] = value

    df_splited_env_values = df.drop([target_col, 'value_types'], axis=1)
    print(df_splited_env_values.columns)
    return df_splited_env_values

def str_to_num(string_list):
    if isinstance(string_list, str):
        try:
            list_obj = ast.literal_eval(string_list)
        except ValueError:
            return string_list
    else:
        list_obj = string_list

    if all(item is None for item in list_obj):
        return None
    else:
        list_num = [float(item) for item in list_obj if item is not None]
        return list_num
    
def string_to_number(df):
    col_list = [col for col in df.columns if col.endswith('_value')]
    
    for col in col_list:
        df[col] = df[col].apply(str_to_num)
        
    return df

def calculate_avg(df):
    col_list = [col for col in df.columns if col.endswith('_value')]

    for value_name in col_list:
        for idx, row in df.iterrows():
            values = row[value_name]
            if values is not None:
                df.at[idx, value_name] = np.mean(values)

            else:
                df.at[idx, value_name] = None

    return df

In [6]:
def data_split(df, target_col):
    exploded_df = df.explode(target_col).reset_index(drop=True)
    exploded_df = pd.concat([exploded_df.drop([target_col], axis=1),
                             exploded_df[target_col].apply(pd.Series)], axis=1)
    
    return exploded_df

In [7]:
def split_bbox(df):
    df['bbox_x'] = df['bbox'].apply(lambda bbox: bbox[0])
    df['bbox_y'] = df['bbox'].apply(lambda bbox: bbox[1])
    df['bbox_width'] = df['bbox'].apply(lambda bbox: bbox[2])
    df['bbox_height'] = df['bbox'].apply(lambda bbox: bbox[3])

    return df

def process_rest_values(df, class_kor, class_en):
    df_cleaned = df.rename(columns={'images.fname':'file_name',
                                    'images.width':'image_width',
                                      'images.height':'image_height',
                                        'images.disease_class':'class',
                                          'images.disease_cause_method':'cause_method'})
    
    df_cleaned['cause_method']=df_cleaned['cause_method'].apply(lambda x: x.replace('칼슘부족','low_Ca'))
    df_cleaned['cause_method']=df_cleaned['cause_method'].apply(lambda x: x.replace('질소부족','low_N'))
    df_cleaned['cause_method']=df_cleaned['cause_method'].apply(lambda x: x.replace('철결핍','leck_Fe'))
    df_cleaned['cause_method']=df_cleaned['cause_method'].apply(lambda x: x.replace('수분제한','water_'))
    df_cleaned['cause_method']=df_cleaned['cause_method'].apply(lambda x: x.replace('주입','_injection'))    
    df_cleaned['cause_method']=df_cleaned['cause_method'].apply(lambda x: x.replace(class_kor,class_en))
    df_cleaned['disease_status']=df_cleaned['disease_status'].apply(lambda x: x.replace('E','Y'))
    df_cleaned['class']=df_cleaned['class'].apply(lambda x: x.replace(class_kor, class_en))

    replace_values = {'잎': 'leaf', '줄기': 'stem', '과실': 'fruit', '화방': 'flower'}
    df_cleaned['name'] = df_cleaned['name'].replace(replace_values, regex=True)
    df_cleaned = df_cleaned.drop(['ir_value', 'tl_value', 'rp_value'], axis=1)
    return df_cleaned

In [8]:
# envrionments 컬럼 데이터 처리 통합 함수

def process_col_environments(df, target_col, saving_col):
    env_target, env_saving = target_col, saving_col
    env_pandas, env_list = json_data_to_pandas(df, target_col=env_target, saving_col=env_saving)
    env_values = extraction_env_values(env_pandas, env_list, env_saving)
    env_values_num = string_to_number(env_values)
    env_values_avg = calculate_avg(env_values_num)
    
    return env_values_avg

In [9]:
# annotations, categories 컬럼 데이터 처리 통합 함수

def process_col_annotations_categories(df, target_col_1, saving_col_1, target_col_2, saving_col_2):
    df_anno = df.drop(['categories'], axis=1)
    df_cat = df[['images.fname', 'categories']].copy()
    anno_target, anno_saving = target_col_1, saving_col_1
    cat_target, cat_saving = target_col_2, saving_col_2

    anno_pandas, _ = json_data_to_pandas(df_anno, anno_target, anno_saving)
    cat_pandas, _ = json_data_to_pandas(df_cat, cat_target, cat_saving)

    anno_split = data_split(anno_pandas, target_col=anno_saving)
    anno_split_sel = anno_split.drop(['coordinates', 'area', 'isCrowd',
                                      'id', 'image_id'],
                                      axis=1)
    cat_split = data_split(cat_pandas, target_col=cat_saving)
    anno_split_sel['category_id'] = cat_split['id']
    anno_split_sel['name'] = cat_split['name']

    return anno_split_sel

In [10]:
def sampling_df(df, ratio):
    total_rows = len(df)
    sampling_rows_num = int(round((total_rows*ratio), -2))
    sampled_df = df.tail(sampling_rows_num)
    sampled_df = sampled_df.reset_index(drop=True)

    return sampled_df

In [11]:
## integrated label preprocess functions

# 모든 대상 컬럼 데이터 처리 통합 함수
    # 컬럼 규칙
        # environments: environments -> env_values -> value_types -> 개별 컬럼들
        # annotations: annotations -> anno_values -> 개별 컬럼들
        # categories: categories -> cat_values -> 개별 컬럼들

def label_preprocess(df, target_0, target_1, target_2, saving_0, saving_1, saving_2, class_kor, class_en):
    processed_env = process_col_environments(df, target_0, saving_0)
    processed_anno_cat = process_col_annotations_categories(processed_env, target_1, saving_1, target_2, saving_2)
    processed_bbox = split_bbox(processed_anno_cat)
    processed_data = process_rest_values(processed_bbox, class_kor, class_en)

    return processed_data

In [14]:
# train data preparation

# normal_preprocessed = label_preprocess(normal_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '정상', 'normal')
# blight_preprocessed = label_preprocess(blight_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '역병', 'blight')
# wilt_preprocessed = label_preprocess(wilt_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '시들음병', 'wilt')
# scorch_preprocessed = label_preprocess(scorch_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '잎끝마름', 'scorch')
# chlorosis_preprocessed = label_preprocess(chlorosis_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '황화', 'chlorosis')

<class 'pandas.core.series.Series'>
Index(['annotations', 'categories', 'images.fname', 'images.width',
       'images.height', 'images.disease_class', 'images.disease_cause_method',
       'folder_path', 'ti_value', 'hi_value', 'ci_value', 'ir_value',
       'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value',
       'hl_value', 'pi_value', 'rp_value'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Index(['annotations', 'categories', 'images.fname', 'images.width',
       'images.height', 'images.disease_class', 'images.disease_cause_method',
       'folder_path', 'ti_value', 'hi_value', 'ci_value', 'ir_value',
       'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value',
       'hl_value', 'pi_value', 'rp_value'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Index(['annotations', 'c

In [27]:
## 처리된 데이터 확인 및 컬럼 선별 추가작업
    ### value 컬럼 선택: ei, pl, el, pi
    ### 컬럼 추가, 변경
        # object_status(추가): class=normal => normal, 나머지 => abnormal
        # disease_status(변경): E => Y로 변경(보류)

selection = ['file_name', 'image_width', 'image_height', 'class', 'cause_method', 'folder_path', 'ei_value', 'pl_value', 'el_value', 'pi_value',
             'bbox', 'category_id', 'disease_status', 'name', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']

# normal = normal_preprocessed[selection].copy()
# blight = blight_preprocessed[selection].copy()
# wilt = wilt_preprocessed[selection].copy()
# scorch = scorch_preprocessed[selection].copy()
# chlorosis = chlorosis_preprocessed[selection].copy()

In [28]:
# normal['object_status'] = 'normal'
# blight['object_status'] = 'abnormal'
# wilt['object_status'] = 'abnormal'
# scorch['object_status'] = 'abnormal'
# chlorosis['object_status'] = 'abnormal'

In [30]:
# train_20 = pd.concat([normal, blight, wilt, scorch, chlorosis], axis=0)

In [31]:
# train_20.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242562 entries, 0 to 27542
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   file_name       242562 non-null  object 
 1   image_width     242562 non-null  int64  
 2   image_height    242562 non-null  int64  
 3   class           242562 non-null  object 
 4   cause_method    242562 non-null  object 
 5   folder_path     242562 non-null  object 
 6   ei_value        242562 non-null  object 
 7   pl_value        242562 non-null  object 
 8   el_value        242562 non-null  object 
 9   pi_value        242562 non-null  object 
 10  bbox            242562 non-null  object 
 11  category_id     242562 non-null  int64  
 12  disease_status  242562 non-null  object 
 13  name            242562 non-null  object 
 14  bbox_x          242562 non-null  float64
 15  bbox_y          242562 non-null  float64
 16  bbox_width      242562 non-null  float64
 17  bbox_height     

In [33]:
# train_20.to_csv('./data_preprocessed/label/train_20.csv', encoding='utf-8', index=False)

In [12]:
# sampling and preprocessed df

col_selection = ['environments', 'annotations', 'categories',
                 'images.fname', 'images.width', 'images.height',
                 'images.disease_class', 'images.disease_cause_method',
                 'folder_path']

final_selection = ['file_name', 'image_width', 'image_height',
                   'class', 'cause_method', 'folder_path',
                   'ei_value', 'pl_value', 'el_value', 'pi_value',
                   'bbox','bbox_x', 'bbox_y', 'bbox_width', 'bbox_height',
                   'category_id', 'disease_status', 'name']

common_args = ['environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values']
class_args = {"normal": ["정상", "normal"], "blight": ["역병", "blight"],
              "wilt": ["시들음병", "wilt"], "scorch": ["잎끝마름", "scorch"],
              "chlorosis": ["황화", "chlorosis"]}

path_str = './data_preprocessed/label/'
name_str = 'train_label'
raw_df_list = [tr_normal, tr_blight, tr_wilt, tr_scorch, tr_chlorosis]


def preprocessing_and_sampling(ratio, df_list, col_selection, final_selection, common_arg_list, class_arg_dict):
    df_sel_list = []
    df_sel_sampling_list = []
    df_sel_final_list = []
    class_args_list = list(class_arg_dict.values())

    for df in df_list:
        df_sel = columns_selection(df, col_selection)
        df_sel_list.append(df_sel)
        
    for df_selected in df_sel_list:
        df_sampled = sampling_df(df_selected, ratio)
        df_sel_sampling_list.append(df_sampled)
        
    for sample, class_args in zip(df_sel_sampling_list, class_args_list):
        df_processed = label_preprocess(sample, *common_arg_list, *class_args)
        df_processed_cols = columns_selection(df_processed, final_selection)
        if df_processed_cols['class'].unique() == 'normal':
            df_processed_cols['object_status'] = 'normal'
        else:
            df_processed_cols['object_status'] = 'abnormal'

        df_sel_final_list.append(df_processed_cols)
        
    df_preprocessed = pd.concat(df_sel_final_list, axis=0)

    return df_preprocessed




In [22]:
# train_label_30 = preprocessing_and_sampling(ratio=0.3, df_list=raw_df_list,
#                                             col_selection=col_selection, final_selection=final_selection,
#                                             common_arg_list=common_args, class_arg_dict=class_args)

ratio_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
ratio_name_list = [20, 30, 40, 50, 60, 70, 80, 90, 100]
arg_list = [raw_df_list, col_selection, final_selection, common_args, class_args]

for ratio in ratio_list:
    name = int(ratio*100)
    saving = f"./data_preprocessed/label/train_label_{name}.csv"
    train_label = preprocessing_and_sampling(ratio, *arg_list)
    train_label.to_csv(saving, encoding='utf-8', index=False)
    print(f"train_label_{name}")

<class 'pandas.core.series.Series'>
Index(['annotations', 'categories', 'images.fname', 'images.width',
       'images.height', 'images.disease_class', 'images.disease_cause_method',
       'folder_path', 'ti_value', 'hi_value', 'ci_value', 'ir_value',
       'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value',
       'hl_value', 'pi_value', 'rp_value'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Index(['annotations', 'categories', 'images.fname', 'images.width',
       'images.height', 'images.disease_class', 'images.disease_cause_method',
       'folder_path', 'ti_value', 'hi_value', 'ci_value', 'ir_value',
       'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value',
       'hl_value', 'pi_value', 'rp_value'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Index(['annotations', 'c

In [34]:
# train_label_30['object_status'].value_counts()

object_status
abnormal    229683
normal      119756
Name: count, dtype: int64

In [19]:
# test data preparation

vl_normal_preprocessed = label_preprocess(vl_normal_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '정상', 'normal')
vl_blight_preprocessed = label_preprocess(vl_blight_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '역병', 'blight')
vl_wilt_preprocessed = label_preprocess(vl_wilt_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '시들음병', 'wilt')
vl_scorch_preprocessed = label_preprocess(vl_scorch_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '잎끝마름', 'scorch')
vl_chlorosis_preprocessed = label_preprocess(vl_chlorosis_data, 'environments', 'annotations', 'categories', 'env_values', 'anno_values', 'cat_values', '황화', 'chlorosis')

<class 'pandas.core.series.Series'>
Index(['annotations', 'categories', 'images.fname', 'images.width',
       'images.height', 'images.disease_class', 'images.disease_cause_method',
       'folder_path', 'ti_value', 'hi_value', 'ci_value', 'ir_value',
       'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value',
       'hl_value', 'pi_value', 'rp_value'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Index(['annotations', 'categories', 'images.fname', 'images.width',
       'images.height', 'images.disease_class', 'images.disease_cause_method',
       'folder_path', 'ti_value', 'hi_value', 'ci_value', 'ir_value',
       'tl_value', 'ei_value', 'pl_value', 'sr_value', 'cl_value', 'el_value',
       'hl_value', 'pi_value', 'rp_value'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Index(['annotations', 'c

In [20]:
selection = ['file_name', 'image_width', 'image_height', 'class', 'cause_method', 'folder_path', 'ei_value', 'pl_value', 'el_value', 'pi_value',
             'bbox', 'category_id', 'disease_status', 'name', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']

vl_normal = vl_normal_preprocessed[selection].copy()
vl_blight = vl_blight_preprocessed[selection].copy()
vl_wilt = vl_wilt_preprocessed[selection].copy()
vl_scorch = vl_scorch_preprocessed[selection].copy()
vl_chlorosis = vl_chlorosis_preprocessed[selection].copy()

vl_normal['object_status'] = 'normal'
vl_blight['object_status'] = 'abnormal'
vl_wilt['object_status'] = 'abnormal'
vl_scorch['object_status'] = 'abnormal'
vl_chlorosis['object_status'] = 'abnormal'

test_label = pd.concat([vl_normal, vl_blight, vl_wilt, vl_scorch, vl_chlorosis], axis=0)

In [21]:
## saving csv files

test_label.to_csv('./data_preprocessed/label/test_label.csv', encoding='utf-8', index=False)

In [2]:
train_label = pd.read_csv('./data_preprocessed/label/train_20.csv', encoding='utf-8')
test_label = pd.read_csv('./data_preprocessed/label/test_label.csv', encoding='utf-8')

In [4]:
train_label['disease_status'].value_counts()

disease_status
N    201457
Y     39058
E      2047
Name: count, dtype: int64

In [5]:
train_label['disease_status']=train_label['disease_status'].apply(lambda x: x.replace('E','Y'))
test_label['disease_status']=test_label['disease_status'].apply(lambda x: x.replace('E','Y'))

In [7]:
test_label['disease_status'].value_counts()

disease_status
N    127849
Y     17545
Name: count, dtype: int64

In [8]:
train_label.to_csv('./data_preprocessed/label/train_20.csv', encoding='utf-8', index=False)
test_label.to_csv('./data_preprocessed/label/test_label.csv', encoding='utf-8', index=False)